In [ ]:
%matplotlib widget
# %matplotlib inline

In [ ]:
from pathlib import Path
import re
from itertools import product
import os
from math import sqrt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from fluidsim import load
from fluidsim.util import (
    get_dataframe_from_paths,
    times_start_last_from_path,
)

In [ ]:
height = 5.5
plt.rc("figure", figsize=(1.33 * height, height))

In [ ]:
path_base = Path("/fsnet/project/meige/2022/22STRATURBANIS/aniso")
if path_base.exists():
    path_base_occigen = path_base.parent / "from_occigen/aniso"
    paths_all = sorted(
        list(path_base.glob(f"ns3d.strat*"))
        + list(path_base_occigen.glob(f"ns3d.strat*"))
    )
elif "STRAT_TURB_TORO2022" in os.environ:
    paths_all = sorted(
        Path(os.environ["STRAT_TURB_TORO2022"]).glob("simul_folders/ns3d*")
    )
else:
    raise Exception(
        "Cannot find the simulation folder. You can set the environment variable STRAT_TURB_TORO2022."
    )


def get_path_finer_resol(N, Rb):
    str_N = f"_N{N}_"
    str_Rb = f"_Rb{Rb:.3g}_"
    str_Rb2 = f"_Rb{Rb}_"
    paths_couple = [
        p
        for p in paths_all
        if str_N in p.name and (str_Rb in p.name or str_Rb2 in p.name)
    ]
    paths_couple.sort(key=lambda p: int(p.name.split("x")[1]), reverse=True)
    for path in paths_couple:
        t_start, t_last = times_start_last_from_path(path)
        if t_last > t_start + 1:
            return path

In [ ]:
from util import couples320

paths = []
for N, Rb in sorted(couples320):
    paths.append(get_path_finer_resol(N, Rb))

[p.name for p in paths]

In [ ]:
print(f"Using {len(paths)} simulations")

In [ ]:
def customize(result, sim):

    EKh = result["EKh"]
    EKz = result["EKz"]
    EK = EKh + EKz
    U = sqrt(2 * EK / 3)
    nu_2 = sim.params.nu_2
    epsK = result["epsK"]

    result["name"] = sim.output.name_run

    result["lambda"] = sqrt(U**2 * nu_2 / epsK)
    result["Re_lambda"] = U * result["lambda"] / nu_2

    result["Rb"] = float(sim.params.short_name_type_run.split("_Rb")[-1])
    result["nx"] = sim.params.oper.nx
    result["nz"] = sim.params.oper.nz


df = get_dataframe_from_paths(
    paths, tmin="t_start+2", use_cache=1, customize=customize
)

In [ ]:
df["Re"] = df.Rb * df.N**2
df["k_max*lambda"] = df["k_max"] * df["lambda"]

columns_old = df.columns.tolist()

# fmt: off
first_columns = [
    "N", "Rb", "Re", "nx", "Fh", "R2", "k_max*eta", "epsK2/epsK", "k_max*lambda", "Gamma", 
    "lx1", "lx2", "lz1", "lz2", "I_velocity", "I_dissipation"]
# fmt: on

columns = first_columns.copy()
for key in columns_old:
    if key not in columns:
        columns.append(key)

df = df[columns]

In [ ]:
df[first_columns]

In [ ]:
df[(df.Fh < 0.04) & (df.R2 > 10)][first_columns]

In [ ]:
def plot(
    df,
    x,
    y,
    logx=True,
    logy=False,
    c=None,
    vmin=None,
    vmax=None,
    s=None,
    clabel=None,
):
    ax = df.plot.scatter(
        x=x,
        y=y,
        logx=logx,
        logy=logy,
        c=c,
        edgecolors="k",
        vmin=vmin,
        vmax=vmax,
        s=s,
    )
    pc = ax.collections[0]
    pc.set_cmap("inferno")
    plt.colorbar(pc, ax=ax)
    if clabel is not None:
        if isinstance(clabel, tuple):
            x, y, clabel = clabel
        ax.figure.text(0.78, 0.05, clabel)
    return ax

## Non-dimensional numbers

In [ ]:
df["R2/Rb"] = df.R2 / df.Rb
plot(
    df,
    "Rb",
    "R2/Rb",
    c=np.log10(df.Fh),
    vmin=-2,
    vmax=-0.8,
    clabel="$\log_{10}(F_h)$",
);

In [ ]:
# df["tmp"] = 1 / df.Fh
# plot(df, "Fh", "tmp", c=np.log10(df.Rb), vmin=1, vmax=2)
# del df["tmp"]

## Hyperviscosity

The order-4 hyper buoyancy Reynolds number is large except for very small Fh. Larger simulations need to be run for $F_h < 10^{-2}$.

In [ ]:
plot(df, "Fh", "R4", logy=True);

In [ ]:
ax = plot(df, "Fh", "Rb", c=df["epsK2/epsK"], logy=True, vmin=0, vmax=1)
ax.set_xlim(right=1)
ax.set_ylim(top=1e3);

In [ ]:
ax = plot(df, "Fh", "R2", c=df["epsK2/epsK"], logy=True, vmin=0, vmax=1)
ax.set_xlim(right=1)
ax.set_ylim(top=1e3);

In [ ]:
ax = plot(df, "Fh", "epsK2/epsK", c=np.log10(df["R2"]), vmin=0.5, vmax=2)
ax.set_ylim(bottom=0);

In [ ]:
df[df["epsK2/epsK"] < 0.6][first_columns]

## Quantification of 2 types of isotropy

### Large scale isotropy `I_velocity`

I_velocity = 3 * EKz / EK

In [ ]:
tmp = df[df.R2 < 120]
ax = plot(tmp, "Fh", "R2", c=tmp.I_velocity, vmin=0.1, vmax=0.4)

In [ ]:
ax = plot(
    df, "Fh", "I_velocity", c=np.log10(df["R2"]), vmin=0.5, vmax=2, logy=True
)
xs = np.linspace(1e-2, 1e-1, 4)
ax.plot(xs, 8e0 * xs**1)
ax.text(0.04, 0.2, "$F_h^{1}$");

In [ ]:
plot(df, "R2", "I_velocity", c=np.log10(df["Fh"]), vmin=-2, vmax=-1);

### Small scale isotropy `I_dissipation`



In [ ]:
tmp = df[df.R2 < 120]
ax = plot(tmp, "Fh", "R2", c=tmp.I_dissipation)

In [ ]:
plot(df, "Fh", "I_dissipation", c=np.log10(df["R2"]), vmin=0.5, vmax=2);

In [ ]:
plot(df, "R2", "I_dissipation", c=np.log10(df["Fh"]), vmin=-2, vmax=-1);

In [ ]:
ax = plot(
    df,
    "Fh",
    "R2",
    c=df["I_velocity"],
    logy=True,
    vmin=0.2,
    vmax=0.8,
    clabel=(0.85, 0.05, r"$I_\mathit{velocity}$"),
    s=50 * df["I_dissipation"],
)
ax.set_xlim(right=1)
ax.set_ylim(top=1e3)
ax.set_xlabel("$F_h$")
ax.set_ylabel("$\mathcal{R} = Re {F_h}^2$")
fig = ax.figure

ax_legend = fig.add_axes([0.15, 0.72, 0.15, 0.12])
ax_legend.set_xticklabels([])
ax_legend.set_xticks([])
ax_legend.set_yticklabels([])
ax_legend.set_yticks([])
isotropy_diss = np.array([0.1, 0.5, 0.9])
heights = np.array([0.2, 0.5, 0.8])
ax_legend.scatter([0.15, 0.15, 0.15], heights, s=50 * isotropy_diss)
ax_legend.set_xlim([0, 1])
ax_legend.set_ylim([0, 1])

for h, i in zip(heights, isotropy_diss):
    ax_legend.text(0.28, h - 0.06, r"$I_\mathit{diss} = " + f"{i}$");

## Mixing coefficient

In [ ]:
def plot2(dataframe, ax=None, color=None):
    return dataframe.plot.scatter(
        x="Fh", y="Gamma", logx=True, ax=ax, color=color
    )


tab10 = plt.get_cmap("tab10")

Ns = sorted(df.N.unique())

ax = None
for iN, N in enumerate(Ns):
    ax = plot2(df[df.N == N], ax=ax, color=tab10(iN % 10))


fig = ax.figure

ax_sub = fig.add_axes([0.6, 0.6, 1.33 * 0.2, 0.2])


def plot2(dataframe, color=None):
    dataframe.plot.scatter(
        x="Fh", y="Gamma", logx=True, logy=True, ax=ax_sub, color=color
    )


for iN, N in enumerate(Ns):
    ax = plot2(df[df.N == N], color=tab10(iN % 10))

ax_sub.set_xlabel("")
ax_sub.set_ylabel("")

xs = np.linspace(7e-2, 3.5e-1, 4)
ax_sub.plot(xs, 3e-2 * xs**-1)
ax_sub.text(0.1, 0.1, "$k^{-1}$")

xs = np.linspace(5e-1, 1.5, 4)
ax_sub.plot(xs, 3e-2 * xs**-2)
ax_sub.text(1, 0.05, "$k^{-2}$");

In [ ]:
ax = plot(
    df,
    "Fh",
    "Gamma",
    c=np.log10(df["R2"]),
    vmin=0.5,
    vmax=2,
    logy=True,
)
xs = np.linspace(1.5e-1, 5e-1, 4)
ax.plot(xs, 5e-2 * xs**-1)
ax.text(0.16, 0.16, "${F_h}^{-1}$")

xs = np.linspace(6e-1, 4, 4)
ax.plot(xs, 5e-2 * xs**-2)
ax.text(1.2, 0.05, "${F_h}^{-2}$");

In [ ]:
ax = plot(df, "Fh", "Gamma", c=np.log10(df["R2"]), vmin=0.5, vmax=2)

In [ ]:
ax = plot(
    df,
    "Fh",
    "R2",
    c=df["Gamma"],
    logy=True,
    vmin=0.2,
    vmax=0.5,
    clabel=(0.8, 0.05, "$\Gamma=\epsilon_A / \epsilon_K$"),
)
ax.set_xlim(right=1)
ax.set_ylim(top=1e3)
ax.set_xlabel("$F_h$")
ax.set_ylabel("$\mathcal{R} = Re {F_h}^2$");

## Scaling laws for averaged quantities

In [ ]:
tmp = df[df.Re > 2000]
ax = plot(tmp, "Fh", "lz1", c=np.log10(tmp["R2"]), vmin=0.5, vmax=2, logy=True)
xs = np.linspace(1e-2, 1.2e-1, 4)

ax.plot(xs, 1e1 * xs**1, "b")
ax.plot(xs, 1.5e0 * xs**0.5, "r");

In [ ]:
tmp = df[df.Re > 2000]
ax = plot(tmp, "Fh", "lx1", c=np.log10(tmp["R2"]), vmin=0.5, vmax=2, logy=True)

In [ ]:
tmp = df[df.Re > 2000].copy()
tmp["lz1/lx1"] = tmp.lz1 / tmp.lx1
ax = plot(
    tmp, "Fh", "lz1/lx1", c=np.log10(tmp["R2"]), vmin=0.5, vmax=2, logy=True
)

xs = np.linspace(1e-2, 4e-2, 2)
ax.plot(xs, 1.8e1 * xs**1, "b")

xs = np.linspace(2e-2, 9.0e-2, 2)
ax.plot(xs, 3.5e0 * xs**0.5, "r");

In [ ]:
tmp = df[df.Re > 2000].copy()
tmp["U_z/U_h"] = np.sqrt(2 * tmp.EKz / tmp.EKh)
ax = plot(
    tmp, "Fh", "U_z/U_h", c=np.log10(tmp["R2"]), vmin=0.5, vmax=2, logy=True
)

xs = np.linspace(1e-2, 5.0e-2, 2)
ax.plot(xs, 1e1 * xs**1, "b")

xs = np.linspace(1e-2, 1.0e-1, 2)
ax.plot(xs, 2e0 * xs**0.5, "r");

In [ ]:
tmp = df[df.Re > 2000].copy()
tmp["E_A/E_K"] = tmp.EA / (tmp.EKh)
ax = plot(
    tmp, "Fh", "E_A/E_K", c=np.log10(tmp["R2"]), vmin=0.5, vmax=2, logy=True
)